## ABOUT
__Author__: Pat McCornack

__Date__: 3/25/24

__Purpose__: Use LANDFIRE data to create sample points to train F40 classification model on. Currently set up to use LF23 data. Outputs a layer of 100,000 points with data extracted from the following rasters: FBFM40, FVT, FVC, FVH, FDST, LF ZONE, and BPS_FRG_NEW. Shapefile is output to sample_points directory with the datetime at which it was generated in the file name. 


__Environment Set Up__: To set up the environment to use arcpy copy the arcgispro-py3 environment to your conda environments folder. For me these were located at:
1. C:\Program Files\ArcGIS\Pro\bin\Python\envs
2. C:\Users\\[username]\AppData\Local\miniconda3\envs\

Simply copy the arcgispro-py3 folder from <1> to <2>.

__NOTE__: The final product should filter points out that are within two cells of another point (i.e. ~60m). This is currently not working in the script and needs to be fixed. 

----

In [ ]:
import os

import arcpy
from arcpy.sa import *

import datetime as dt

# Set up arcpy environment
arcpy.env.overwriteOutput=True


# Create sample point layer
Note that CreateAccuracyAssessmentPoints will not run if there are -9999 values, which is those points are first reclassified to NODATA in the F40 raster. 

In [ ]:
# Define paths
## Define geodatabase/directory with source rasters
#gdb = r"C:\Users\mcco573\OneDrive - PNNL\Documents\_Projects\BPA Wildfire\F40 Random Forest Model\data\LF23_data.gdb"
data_dir = r"C:\Users\mcco573\OneDrive - PNNL\Documents\_Projects\BPA Wildfire\LF_raster_data\bpa_service_territory"

## Define name of output sample points file
datetime = dt.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
out_dir = "C:\Users\mcco573\OneDrive - PNNL\Documents\_Projects\BPA Wildfire\F40_modeling\data\sample_points"
out_fname = "F22_sample_points_{datetime}.shp"

out_fpath = os.path.join(out_dir, out_fname)

# Set arcpy workspace
arcpy.env.workspace = data_dir

# Define the number of sample points to create
num_points = 100000

# Define dictionary of layers
raster_paths = {
    "F40" : "LC22_F40_220_bpa.tif",
    "FVT" : "LC22_FVT_220_bpa.tif",
    "FVC" : "LC22_FVC_220_bpa.tif",
    "FVH" : "LC22_FVH_220_bpa.tif",
    "FDST" : "LC22_FDst_220_bpa.tif",
    "ZONE" : "us_lf_zones_bpa.tif",
    "BPS_FRG_NEW" : "BPS_FRG_bpa.tif"
}

# Create list of rasters to extract from - "Value" is the field name to be extracted
raster_list = []
for key, value in raster_paths.items():
    raster_list.append([value, key])

# Check out spatial analyst license if available
if arcpy.CheckExtension("Spatial") == "Available":
    arcpy.CheckOutExtension("Spatial")

    # Reclassify values to NODATA in order to run CreateAccuracyAssessmentPoints
    F40_reclass = Reclassify(in_raster=raster_paths['F40'],
                             reclass_field='Value',
                             remap=RemapValue([[-9999, "NODATA"], [124, "NODATA"], [148, "NODATA"]]))
    #F40_reclass.save("F40_reclass_temp")
    print("Reclass Executed.")

    # Generate Sample Points
    CreateAccuracyAssessmentPoints(in_class_data=F40_reclass,
                                   out_points=out_fpath,
                                   target_field='CLASSIFIED',
                                   num_random_points=num_points,
                                   sampling="STRATIFIED_RANDOM") 
    print("CreateAccuracyAssessmentPoints Executed.")

    # Use near tool to filter points less than 70m apart
    arcpy.analysis.Near(in_features=out_fpath,
                        near_features=out_fpath,
                        search_radius=70,
                        distance_unit="meters")
    print("Near Executed.")

    # Remove points where NEAR_FID != -1
    ## Note that this currently isn't working
    """ sample_points = arcpy.management.SelectLayerByAttribute(in_layer_or_view=out_fpath, 
                                        selection_type="NEW_SELECTION",
                                        where_clause='"NEAR_FID" = -1')
    print("Near Points Filtered.")     """

    # Remove the fields generated by Near
    """ arcpy.management.DeleteField(in_table=out_fpath,
                                 drop_field=['GrndTruth', 'NEAR_FID', 'NEAR_DIST'])     """                                     
    
    # Extract raster values to generated points
    ExtractMultiValuesToPoints(in_point_features=out_fpath,
                               in_rasters=raster_list)
    print("ExtractMultiValuesToPoints Executed.")

# Check the extension back in    
if arcpy.CheckInExtension("Spatial") == "CheckedIn":
    print("Spatial Analyst license successfully checked in.")
else: 
    print("Failed to check in Spatial Analyst license")
